# Prerequisites

The following steps must be satisfied before running the rest of the notebook:

- Run the `Augment_Data` notebook in order to generate the training files and images that are referred to throughout the notebook.
- Make changes to `hm_convert` to read the added files/folders when running `build_dataset`.

Throughout the notebook, directory paths are used for references that depend on your local setup. There are comments that describe what each path should be pointing to so that should help set up the notebook to work for your hierarchy.

# Setup

In [1]:
### Mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# local mmf installation
%cd '/content/drive/MyDrive/hateful-memes/mmf-main'
!pip install -e .

/content/drive/MyDrive/hateful-memes/mmf-main
Obtaining file:///content/drive/MyDrive/hateful-memes/mmf-main
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.9/869.9 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
### Import General Modules, 3min
from mmf.common.registry import registry
from mmf.models.mmbt import MMBT
from mmf.utils.build import build_dataset
from mmf.utils.env import setup_imports
import matplotlib.pyplot as plt
from PIL import Image
import torch

### Import Modules for Fusion Model
# All model using MMF need to inherit BaseModel
from mmf.models.base_model import BaseModel
# Builder methods for image encoder and classifier
from mmf.utils.build import build_classifier_layer, build_image_encoder,build_text_encoder

setup_imports()

# !mmf_convert_hm --zip_file="/content/drive/MyDrive/hateful-memes/hateful_memes.zip" --bypass_checksum=1
# dataset = build_dataset("hateful_memes")

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [2]:
### Build Dataset
# provide the path to the compressed dataset folder (should have hateful_memes as root folder when uncompressed)
!mmf_convert_hm --zip_file="/content/drive/MyDrive/Deep Learning Final Project Part 2/hateful_memes.zip" --bypass_checksum=1
dataset = build_dataset("hateful_memes")

2024-04-24 17:17:47.906276: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 17:17:47.906331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 17:17:47.908242: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-24 17:17:48.966906: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-24 17:17:53.173737: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

/usr/local/lib/python3.10/dist-packages/omegaconf/grammar_visitor.py:253: UserWarning: In the sequence `MMF_LOG_DIR,` some elements are missing: please replace them with empty quoted strings. See https://github.com/omry/omegaconf/issues/572 for details.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/omegaconf/grammar_visitor.py:253: UserWarning: In the sequence `MMF_REPORT_DIR,` some elements are missing: please replace them with empty quoted strings. See https://github.com/omry/omegaconf/issues/572 for details.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/omegaconf/grammar_visitor.py:253: UserWarning: In the sequence `MMF_TENSORBOARD_LOGDIR,` some elements are missing: please replace them with empty quoted strings. See https://github.com/omry/omegaconf/issues/572 for details.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/omegaconf/grammar_visitor.py:253: UserWarning: In the sequence `MMF_WANDB_LOGDIR,` some elements are missing: please replace them wit

[ Downloading: https://dl.fbaipublicfiles.com/mmf/data/datasets/hateful_memes/defaults/extras.tar.gz to /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/extras.tar.gz ]


[ Starting checksum for extras.tar.gz]
[ Checksum successful for extras.tar.gz]
Unpacking extras.tar.gz



/root/.cache/torch/mmf/glove.6B.zip: 0.00B [02:09, ?B/s]


URLError: <urlopen error [Errno 110] Connection timed out>

# Training

The sections below rely on training `json` files that were created in a separate notebook in the Prerequisites section. If you see a warning about these files not existing please make sure to follow the steps in that section.

## Pre-trained with COCO

### Dataset: Default

In [ ]:
!mmf_run config="./configs/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.max_updates=1500 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_zoo=visual_bert.pretrained.coco \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-21 16:00:05.819493: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 16:00:05.819544: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 16:00:05.821140: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-21 16:00:06.829005: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-21 16:00:11.067734: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

### Dataset: Appended with images from dev

In [ ]:
!mmf_run config="./configs/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_zoo=visual_bert.pretrained.coco \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train_dev.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-21 23:16:01.895927: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 23:16:01.895971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 23:16:01.897544: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-21 23:16:02.913111: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-21 23:16:07.044321: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

### Dataset: Augmented with partial transformations

In [ ]:
!mmf_run config="./configs/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_zoo=visual_bert.pretrained.coco \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train_dev_transform_half.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-22 13:56:53.939928: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 13:56:53.939983: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 13:56:53.941702: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 13:56:55.124889: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-22 13:57:01.451812: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

### Dataset: Augmented with complete transformations

In [3]:
!mmf_run config="./configs/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_zoo=visual_bert.pretrained.coco \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train_dev_transform_all.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-24 17:28:37.229214: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 17:28:37.229279: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 17:28:37.230849: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-24 17:28:38.333522: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-24 17:28:43.061966: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

### Dataset: Augmented with combined transformations

In [ ]:
!mmf_run config="./configs/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_zoo=visual_bert.pretrained.coco \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train_dev_transform_all_combined.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-22 14:31:35.365915: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 14:31:35.365966: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 14:31:35.367158: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 14:31:36.432665: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-22 14:31:40.776437: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

## Pretrained with VizWiz

### Dataset: Default

In [ ]:
!mmf_run config="./configs/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.max_updates=1500 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_file="/content/drive/MyDrive/hateful-memes/Deep Learning Group Project/visualBERT_vizwiz_04212024_pretrained.ckpt" \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-23 23:21:05.146836: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 23:21:05.146890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 23:21:05.148719: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 23:21:06.248358: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-23 23:21:10.895631: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

### Dataset: Appended with images from dev

In [ ]:
!mmf_run config="./configs/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_file="/content/drive/MyDrive/hateful-memes/Deep Learning Group Project/visualBERT_vizwiz_04212024_pretrained.ckpt" \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train_dev.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-24 01:47:01.546861: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 01:47:01.546925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 01:47:01.548643: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-24 01:47:02.655882: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-24 01:47:07.215852: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

### Dataset: Augmented with partial transformations

In [ ]:
!mmf_run config="./configs/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_file="/content/drive/MyDrive/hateful-memes/Deep Learning Group Project/visualBERT_vizwiz_04212024_pretrained.ckpt" \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train_dev_transform_half.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-24 02:24:27.641435: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 02:24:27.641493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 02:24:27.642945: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-24 02:24:28.788093: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-24 02:24:34.490252: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

### Dataset: Augmented with complete transformations

In [4]:
!mmf_run config="./configs/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_file="/content/drive/MyDrive/hateful-memes/Deep Learning Group Project/visualBERT_vizwiz_04212024_pretrained.ckpt" \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train_dev_transform_all.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-24 18:28:36.744362: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 18:28:36.744436: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 18:28:36.746058: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-24 18:28:37.854011: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-24 18:28:42.621793: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

### Dataset: Augmented with combined transformations

In [ ]:
!mmf_run config="./configs/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_file="/content/drive/MyDrive/hateful-memes/Deep Learning Group Project/visualBERT_vizwiz_04212024_pretrained.ckpt" \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train_dev_transform_all_combined.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-24 02:58:01.974874: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 02:58:01.974946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 02:58:01.976370: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-24 02:58:03.029554: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-24 02:58:07.448445: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10